# Assignment 1 - Hours Play prediction

## Import libraries

In [25]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
from sklearn import linear_model
import numpy as np
import string
import random
import matplotlib.pyplot as plt
import tensorflow as tf

## Utility functions and data structures

In [277]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [278]:
def readJSON(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        d = eval(l)
        u = d['userID']
        g = d['gameID']
        yield u,g,d

In [279]:
allHours = []
for l in readJSON("train.json.gz"):
    allHours.append(l)

In [280]:
hoursTrain = allHours[:165000]
hoursValid = allHours[165000:]

In [281]:
hoursPerUser = defaultdict(list)
hoursPerItem = defaultdict(list)
for u,g,d in hoursTrain:
    r = d['hours_transformed']
    hoursPerUser[u].append((g, r))
    hoursPerItem[g].append((u, r))

In [282]:
itemsPerUser = defaultdict(list)
usersPerItem = defaultdict(list)
for u,g,d in hoursTrain:
    itemsPerUser[u].append(g)
    usersPerItem[g].append(u)

In [283]:
itemsPerUser = defaultdict(list)
usersPerItem = defaultdict(list)
for u,g,d in hoursTrain:
    itemsPerUser[u].append(g)
    usersPerItem[g].append(u)

In [284]:
userIDs, itemIDs = {}, {}
interactions = []

for u,g,d in allHours:
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not g in itemIDs: itemIDs[g] = len(itemIDs)
    label = 1 # they are positive labels we presume --> might improve by sentiment analysis on text data
    interactions.append((u,g,1))

In [285]:
userIDs = {}
itemIds = {}
interactions = []

for u,g,d in allHours:
    r = d['hours_transformed']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not g in itemIds: itemIDs[g] = len(itemIDs)
    interactions.append((u,g,r))

random.shuffle(interactions)
len(interactions)

174999

In [286]:
nTrain = int(len(interactions)*0.9)
nTest = len(interactions) - nTrain
interactionsTrain = interactions[:nTrain]
interactionsTest = interactions[nTrain:]

In [287]:
interactionsTrain = interactions[:165000]
interactionsTest = interactions[165000:]

In [288]:
# Generate a negative set
userSet = set()
gameSet = set()
playedSet = set()

for u,g,d in allHours:
    userSet.add(u)
    gameSet.add(g)
    playedSet.add((u, g))

In [289]:
len(userSet), len(gameSet)

(6710, 2437)

In [290]:
lUserSet = list(userSet)
lGameSet = list(gameSet)
userPassed = set()

notPlayed = set()
for u,g,d in hoursValid:
    g = random.choice(lGameSet)
    while (u,g) in playedSet or (u,g) in notPlayed:
        g = random.choice(lGameSet)
    notPlayed.add((u,g))

playedValid = set()
for u,g,r in hoursValid:
    playedValid.add((u,g))

In [291]:
import math

## Fine Tuning

BPR
* Learning rate for Adam. --> Done. 0.1 is the best one.
* Try out SGD and fine-tune the parameter --> Sucks, gone
* Number of Epoch to train the model. --> 200.
* Fine-tune the lambda -- maybe split into two lambdas. --> Done. Stick with the original one.
* Threshold for label prediction.
* Fine-tune the K-value of the BPR.
* Ensemble different model predict (incorporate jaccard maybe?).

LFM

* Optimizer choice (Adam or SGD).
* Learning rate for the optimizer.
* Introduce two gamma terms.
* K value (less than 5).
* Alpha (offset term).
* Regularizer (introduce two regularizers instead of one).

In [318]:
userIDs = {}
itemIDs = {}
interactions = []

for u,g,d in allHours:
    r = d['hours_transformed']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not g in itemIDs: itemIDs[g] = len(itemIDs)
    interactions.append((u,g,r))

In [591]:
saved_betaI=tf.identity(model.betaI)
saved_betaU=tf.identity(model.betaU)
saved_gammaI=tf.identity(model.gammaI)
saved_gammaU=tf.identity(model.gammaU)

In [608]:
saved_betaI

<tf.Tensor: shape=(2437,), dtype=float32, numpy=
array([-0.30229905, -1.3742098 ,  0.9545139 , ..., -0.6790927 ,
       -1.3271716 , -2.2350788 ], dtype=float32)>

In [609]:
# mean rating, just for initialization
mu = sum([r for _,_,r in interactionsTrain]) / len(interactionsTrain)

In [610]:
# Gradient descent optimizer, experiment with learning rate
optimizer = tf.keras.optimizers.Adam(0.01)

class LatentFactorModel(tf.keras.Model):
    def __init__(self, mu, K, lamb1, lamb2):
        super(LatentFactorModel, self).__init__()
        # Initialize to average
        self.alpha = tf.Variable(mu)
        # Initialize to small random values
        # self.betaU = tf.Variable(tf.random.normal([len(userIDs)],stddev=0.001))
        # self.betaI = tf.Variable(tf.random.normal([len(itemIDs)],stddev=0.001))
        # self.gammaU = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001))
        # self.gammaI = tf.Variable(tf.random.normal([len(itemIDs),K],stddev=0.001))
        self.betaU = saved_betaU
        self.betaI = saved_betaI
        self.gammaU = saved_gammaU
        self.gammaI = saved_gammaI
        self.lamb1 = lamb1
        self.lamb2 = lamb2

    # Prediction for a single instance (useful for evaluation)
    def predict(self, u, i):
        p = self.alpha + self.betaU[u] + self.betaI[i] +\
            tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p

    # Regularizer
    def reg(self):
        return self.lamb1 * (tf.reduce_sum(self.betaU**2) +\
                           tf.reduce_sum(self.betaI**2)) +\
                self.lamb2 * (tf.reduce_sum(self.gammaU**2) +\
                           tf.reduce_sum(self.gammaI**2))
    
    # Prediction for a sample of instances
    def predictSample(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_u = tf.nn.embedding_lookup(self.betaU, u)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        pred = self.alpha + beta_u + beta_i +\
            tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return pred
    
    # Loss
    def call(self, sampleU, sampleI, sampleR):
        pred = self.predictSample(sampleU, sampleI)
        r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
        return tf.nn.l2_loss(pred - r) / len(sampleR)

In [611]:
# Experiment with number of factors and regularization rate
model = LatentFactorModel(mu, 3, 0.000005, 1.5)

In [612]:
def trainingStep(interactions):
    Nsamples = 50000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleR = [], [], []
        for _ in range(Nsamples):
            u,i,r = random.choice(interactions)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleR.append(r)

        loss = model(sampleU,sampleI,sampleR)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

LR | K | LAMB1 | LAMB2 | EPOCH | MSE

**0.01 | 3 | 0.00001 | 0.001 | 500 | 3.0901693570116437**

**0.01 | 3 | 0.00001 | 0.001 | 800 | 3.067797038747282**

0.01 | 3 | 0.00001 | 0.001 | 400 | 3.11296272467422

0.01 | 2 | 0.00001 | 0.001 | 400 | 3.110882067066202

0.01 | 4 | 0.00001 | 0.001 | 400 | 3.118307844312977

0.01 | 2 | 0.00001 | 0.001 | 500 | 3.118307844312977

**0.1 | 3 | 0.00001 | 0.001 | 400 | 3.11237270120542**

0.1 | 3 | 0.00001 | 0.001 | 500 | 3.107369001485266

In [613]:
def MSE(preds, labels):
    diff = [(x-y)**2 for x,y in zip(preds, labels)]
    return sum(diff) / len(diff)

lambda2=1.5 works the best now.

In [614]:
test_labels = [r for _,_,r in interactionsTest]

In [615]:
# 200 iterations of gradient descent with warm start
# mse_hist = []
for i in range(200):
    obj = trainingStep(interactionsTrain)
    if i % 20 == 19:
        valid_labels = [model.predict(userIDs[u], itemIDs[g]).numpy() for u,g,r in interactionsTest]
        valid_mse = MSE(test_labels, valid_labels)
        print("iteration " + str(i+1) + ", objective = " + str(obj) + ", MSE = " + str(valid_mse))

iteration 20, objective = 1.4300528, MSE = 3.041966680812311
iteration 40, objective = 1.3989687, MSE = 3.0411397828153905
iteration 60, objective = 1.4019612, MSE = 3.0411402101872165
iteration 80, objective = 1.4241041, MSE = 3.0411794279581033


KeyboardInterrupt: 

In [ ]:
3.039763017293007

In [577]:
test_preds = [model.predict(userIDs[u], itemIDs[g]).numpy() for u,g,r in interactionsTest]

In [578]:
MSE(test_labels, test_preds)

3.0486554908572128

In [543]:
len(interactionsTrain)

165000

In [217]:
len(playedValid), len(notPlayed)

(9999, 9999)

In [443]:
test_labels = [r for _,_,r in interactionsTest]

In [489]:
test_preds = [model.predict(userIDs[u], itemIDs[g]).numpy() for u,g,r in interactionsTest]

In [490]:
MSE(test_labels, test_preds)

3.0764907361880964

In [616]:
predictions = open("predictions_Hours.csv", 'w')
for l in open("pairs_Hours.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    pred = model.predict(userIDs[u], itemIDs[g]).numpy()
    _ = predictions.write(u + ',' + g + ',' + str(pred) + '\n')

predictions.close()